## 목표
✅ 4주차: 비동기 처리 & 코루틴
- 코루틴 기본 개념 (suspend, async/await, launch)
- Flow와 Channel 활용
- 실습: 간단한 비동기 API 호출 및 데이터 처리

## 코루틴

- 비동기 처리를 쉽게 해주는 코틀린의 방법
- 쓰레드가 아님(개념상 경량 쓰레드)
- launch / async 로 시작


In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.9.0")
import kotlinx.coroutines.*

In [2]:
runBlocking {
    launch {
        println("🍔 햄버거 만들기 시작!")
        delay(1000)
        println("🍔 햄버거 완성!")
    }
    println("🎮 게임 계속하기")
}

🎮 게임 계속하기
🍔 햄버거 만들기 시작!
🍔 햄버거 완성!


### async / await

- async 는 launcher 와 달리 응답값을 받고 await 로 수신

In [7]:
runBlocking {
    val result = async {
        delay(1000)
        100 + 200
    }
    println("점수 계산중")
    println("점수: ${result.await()} 점")
}


점수 계산중
점수: 300 점
result: async coroutine result
launch


In [8]:
runBlocking {
    launch {
        delay(1000)
        println("launch")
    }

    val deffered = async {
        delay(500)
        "async coroutine result"
    }
    println("result: ${deffered.await()}")
}

result: async coroutine result
launch


## suspend

- 코루틴만 사용할수 있는 함수
- launch / async 에서만 쓸수 있음

In [14]:
suspend fun fetchData(): String {
    delay(500)
    return "API result"
}

runBlocking {
    launch {
        val result = fetchData()
        println("fetch API: $result")
    }
    println("hello")
}


hello
fetch API: API result


## flow

- 자바의 스트림과 비슷함, 연속된 데이터의 비동기 흐름.
- collect() 호출시점에 데이터 생성

In [5]:
import kotlinx.coroutines.flow.*

runBlocking {
    flow {
        emit(1)
        emit(2)
        emit(1)
    }.collect {
        println("물고기 잡았다. $it 마리")
    }
}

물고기 잡았다. 1 마리
물고기 잡았다. 2 마리
물고기 잡았다. 1 마리


### Channel

- 느낌은 MQ
- 코루틴간의 통신을 위한 큐 (소비되면 다시 소비 불가)


In [9]:
import kotlinx.coroutines.channels.*
import javax.naming.InitialContext


runBlocking {
    val channel = Channel<Int>()

    launch {
        for (x in 1..3) {
            println("send data: $x")
            channel.send(x)
        }
        channel.close()
    }

    for (x in channel) {
        println("received: $x")
    }
}

send data: 1
send data: 2
received: 1
received: 2
send data: 3
received: 3


### chunk

- 컬렉션을 단위로 나누는것

In [15]:
val chunkedList = (1..10).chunked(3)
println(chunkedList)

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10]]


### semaphore

- 코루틴에서 한번에 수행할수 있는갯수

In [17]:
import kotlinx.coroutines.sync.*

runBlocking {
    val semaphore: Semaphore = Semaphore(2)

    val jobs = List(5) { i ->
        launch {
            semaphore.withPermit {
                println("작업 $i 시작")
                delay(1000)
                println("작업 $i 종료")
            }
        }
    }
    jobs.joinAll()
}

작업 0 시작
작업 1 시작
작업 0 종료
작업 1 종료
작업 2 시작
작업 3 시작
작업 2 종료
작업 3 종료
작업 4 시작
작업 4 종료


### Dispatcher

- 코루틴을 어떤 쓰레드에서 수행할것인지 정해주는 것
- Default(CPU 연산 최적화) / IO / Main (안드로이드 UI 쓰레드) / Unconfined (제한없이 현재 쓰레드 사용)


In [24]:
import kotlinx.coroutines.*

runBlocking {
    launch(Dispatchers.Default) {
        println("Default 작업: ${Thread.currentThread().name}")
    }
    launch(Dispatchers.IO) {
        println("IO 작업: ${Thread.currentThread().name}")
    }
}

Default 작업: DefaultDispatcher-worker-3
IO 작업: DefaultDispatcher-worker-2
Unconfined 작업: Execution of code 'import kotlinx.corou...'


StandaloneCoroutine{Completed}@40eb48d